In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from scipy.stats import f

import statsmodels.api as sm
import seaborn as sns

import matplotlib.pyplot as plt
import os

**SCARICA IL DATASET**

In [ ]:
file_name = "Holstein_diary_cows.csv"
url = "https://www.dropbox.com/scl/fi/r5sfzpe7oiyup8delhqcw/Holstein_diary_cows.csv?rlkey=tbnxs2z05osci2k6c06wo17av&st=ggok5pnp&dl=0"
# Download the file using wget
os.system(f"wget -O {file_name} '{url}'")

print(f"File '{file_name}' scaricato nella directory corrente.")

**FUNZIONI PER VALUTARE LE PREDIZIONI SUL TEST SET**

In [ ]:
def evaluate_predictions(X_test, y_test, y_pred):
    # R-squared (R²)
    r2 = r2_score(y_test, y_pred)

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Residual Standard Error (RSE)
    # Residuals are the differences between the true values and the predictions
    residuals = y_test - y_pred
    # For simple linear regression, degrees of freedom = n - 2
    rse = np.sqrt(np.sum(residuals**2) / (len(y_test) - 2))

    # Output all the results
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}%")
    print(f"Residual Standard Error (RSE): {rse:.4f}")
    # Create the scatter plot
    plt.scatter(X_test, y_test, color='blue', marker='o',
                label='ground truth BHB values')
    plt.scatter(X_test, y_pred, color='red', marker='+',
                label='predicted BHB values')
    plt.axhline(y=1.2, color='green', linestyle='--',
                linewidth=2, label='Threshold per diagnosi')
    # Set labels and title
    plt.xlabel('Glu (mmol/L)')
    plt.ylabel('BHB (mmol/L)')
    plt.title('Scatter plot: ground truth and predictions on test set')

    # Add a legend
    plt.legend()

    # Display the plot
    plt.show()

    # Create a histogram
    plt.figure(figsize=(8, 6))  # Adjust figure size if needed
    # kde=True adds a kernel density estimate
    sns.histplot(residuals, kde=True, bins=30)
    plt.title('Distribuzione dei residui')
    plt.xlabel('Residuai')
    plt.ylabel('Frequenza')
    plt.show()

    # Create a Q-Q plot (optional)
    sm.qqplot(residuals, line='45', fit=True)
    plt.title('Q-Q Plot dei residui')
    plt.show()


# Function to compute TP, TN, FP, FN
def compute_confusion_matrix(boolean_predictions, boolean_ground_truth):
    TP = np.sum((boolean_predictions == True) & (
        boolean_ground_truth == True))   # Both True
    TN = np.sum((boolean_predictions == False) & (
        boolean_ground_truth == False))  # Both False
    # Predicted True, but False in ground truth
    FP = np.sum((boolean_predictions == True) &
                (boolean_ground_truth == False))
    # Predicted False, but True in ground truth
    FN = np.sum((boolean_predictions == False) &
                (boolean_ground_truth == True))
    return TP, TN, FP, FN


def compute_classification_metrics(TP, TN, FP, FN):
    """
    Computes accuracy, recall, and F1 score.

    Args:
      TP: True positives.
      TN: True negatives.
      FP: False positives.
      FN: False negatives.

    Returns:
      A tuple containing accuracy, recall, and F1 score.
    """
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return accuracy, recall, precision, f1_score

Cliccare sull'icona della cartella qui a sinistra. Quando si apre lo spazio di lavoro dei file, trascinare il file 'Holstein_diary_cows.csv' dal proprio computer all'area qui a sinistra. In seguito eseguire la prossima cella.

In [ ]:
cvs_source= 'Holstein_diary_cows.csv'
df = pd.read_csv(cvs_source)

In [ ]:
# @title Eseguire questa cella per creare il dataset solo se non si è riusciti a caricare il file csv dal proprio computer
# Set random seed for reproducibility
np.random.seed(42)

# Generate X values in the range 1.8–4.5
X_0 = np.random.uniform(1.8, 4.5, 30)
mu = (2.29 + 2.462) / 2
sigma = 0.1
X_1 = np.random.normal(loc=mu, scale=sigma, size=30)
mu = (1.8 + 4.5) / 2
sigma = 1.5
X_2 = np.random.normal(loc=mu, scale=sigma, size=110)
mu = (2.29 + 2.462) / 2
sigma = 1.0
X_3 = np.random.normal(loc=mu, scale=sigma, size=60)
X = np.concatenate((X_0, X_1, X_2, X_3))
X = X[X < 4.5]
X = X[X > 1.8]
# Sort the X values in ascending order
X.sort()
# Define the parameters for the linear relation Y = aX + B
a = 0.220
b = -1.98
c = 4.655

Y = a*(X**2) + b*X + c

# Calculate Y values
Y_noisy = np.zeros_like(Y)

# Add Gaussian noise while ensuring non-negativity
for i in range(len(Y)):
    noise = np.random.normal(0, 0.1)
    v = Y[i] + noise
    if v <= 0:
        Y_noisy[i] = Y[i] + np.random.normal(0, 0.000001)
    else:
        Y_noisy[i] = v

# Create the pandas DataFrame
df = pd.DataFrame({'Glu': X, 'BHB': Y_noisy})

**Eseguire la prossima cella per visualizzare il dataset**

In [ ]:
# Create the scatter plot
plt.scatter(df['Glu'], df['BHB'])

# Set labels and title
plt.xlabel('Glu (mmol/L)')
plt.ylabel('BHB (mmol/L)')
plt.title('Scatter plot: Glu vs BHB')

# Add a legend
plt.legend()

# Display the plot
plt.show()

**DIVIDI IL DATASET IN 80% TRAIN E 20% TEST**

In [ ]:
X = df[['Glu']]  # Features (independent variable)
y = df['BHB']    # Target (dependent variable)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**CREA ED ALLENA UN MODELLO DI REGRESSIONE LINEARE**

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

slope = model.coef_[0]
intercept = model.intercept_

print(f"Slope: {slope}")
print(f"Intercept: {intercept}")

**VISUALIZZA LA RETTA DI REGRESSIONE ED APPLICA LA F-STATISTIC**

In [ ]:
# Create the scatter plot
plt.scatter(df['Glu'], df['BHB'])
# Generate x-values for the regression line
x_line = np.array([df['Glu'].min(), df['Glu'].max()])

# Calculate y-values for the regression line using slope and intercept
y_line = slope * x_line + intercept
# Plot the regression line
plt.plot(x_line, y_line, color='red', label='Retta di regressione')
# Set labels and title
plt.xlabel('Glu (mmol/L)')
plt.ylabel('BHB (mmol/L)')
plt.title('Scatter plot: Glu vs BHB')
# Add a legend
plt.legend()
# Display the plot
plt.show()

# F-statistic (based on the regression model)
# We will use statsmodels to compute the F-statistic for the regression model
# Add constant (intercept) to X_test
X_test_with_const = sm.add_constant(X_test)
# Fit the model using statsmodels
model_sm = sm.OLS(y_test, X_test_with_const).fit()
f_statistic = model_sm.fvalue  # Extract F-statistic
print(f"F-statistic: {f_statistic:.4f}")

df1 = 1  # Numerator degrees of freedom
df2 = 172  # Denominator degrees of freedom

# Compute the p-value
p_value = f.sf(f_statistic, df1, df2)  # Survival function: P(F >= F_stat)
print(f"P-value: {p_value:.20f}")

**APPLICA IL MODELLO SUL TEST SET E VALUTA LE SUE PREDIZIONI**

In [ ]:
y_pred = model.predict(X_test)
evaluate_predictions(X_test, y_test, y_pred)

if not isinstance(y_test, np.ndarray):
  y_test = y_test.values

threshold = 1.2  # The threshold value

# Create boolean arrays
bool_y_test = y_test >= threshold
bool_y_pred = y_pred >= threshold

# Compute metrics for each prediction array
TP, TN, FP, FN = compute_confusion_matrix(bool_y_pred, bool_y_test)
accuracy, recall, precision, f1_score = compute_classification_metrics(
    TP, TN, FP, FN)

print("Matrice di confusione:")
print(f"TP: {TP}")
print(f"TN: {TN}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print("Metriche del classificatore:")
print(f"Accuratezza: {accuracy}")
print(f"Richiamo: {recall}")
print(f"Precisione: {precision}")
print(f"F1: {f1_score}")